In [1]:
import pickle
import torch

from transformers import AutoTokenizer

from utils import Passage_Embedding
from encoder import BertEncoder_For_BiEncoder, RoBertaEncoder_For_CrossEncoder
from rerank import rerank, get_relavant_doc

In [2]:
BertEncoder = BertEncoder_For_BiEncoder
q_encoder = torch.load('./assets/q_encoder.pt', map_location=torch.device('cpu'))

with open('./assets/passage_embedding.bin', 'rb') as file :
    p_embs = pickle.load(file)

wiki_path = './assets/wikipedia_documents.json' # put in your own wiki data path
corpus = Passage_Embedding(wiki_path, p_encoder=None).get_corpus()

RoBertaEncoder = RoBertaEncoder_For_CrossEncoder
c_encoder = torch.load("./assets/c_encoder.pt", map_location=torch.device('cpu'))

In [5]:
query = '나폴레옹이 죽은 날짜는?'
query = [query]

['나폴레옹이 죽은 날짜는?']

In [16]:
model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

doc_scores, doc_indices = get_relavant_doc(q_encoder, tokenizer, query, p_embs, k = 10)

In [18]:
# Cross-Encoder Retrieval (Re-Ranking)
result_scores, result_indices = rerank(query, c_encoder, doc_indices, corpus, tokenizer)

  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\user\Desktop\project\cross_encoder\rerank.py:126: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(
c:\Users\user\Desktop\project\cross_encoder\rerank.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(
c:\Users\user\Desktop\project\cross_encoder\rerank.py:132: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_type_ids = torch.tensor(
100%|██████████| 1/1 [00:19<00:00, 19.26s/it]


In [19]:
# get final Top-k Passages: Here, I just get 50 passage
final_indices = []
for i in range(len(doc_indices)):
    t_list = [doc_indices[i][result_indices[i][k]] for k in range(5)]
    final_indices.append(t_list)

In [20]:
corpus[final_indices[0][0]]

'1796년, 나폴레옹의 이탈리아 원정 때 전속부관이 되었다. 이탈리아 원정과 이집트 원정에서 계속해서 큰 공을 세웠으며, 브뤼메르 18일 쿠데타때 중요한 몫을 맡아 성공적으로 수행했다 . 이 쿠데타의 성공으로 제1집정이 된 나폴레옹은 누이동생 카롤린을 뮈라와 혼인시켰다 .\n\n1800년, 마렝고 전투를 승리로 이끈데 이어 1801년 부르봉 왕가가 다스리던 나폴리 왕국에 폴리뇨 조약을 강요하여 이탈리아 반도를 제압했다. 1804년 12월 2일, 나폴레옹이 제위에 오르자 뮈라는 원수로 승진했다.\n\n1805년, 울름에서 오스트리아군의 발을 묶은 후 항복시킴으로써 아우스터리츠 전투에서 결국 오스트리아와 러시아가 패전하는 결정적 요인을 제공했다. 이듬해인 1806년에는 예나 전투에서 기병대를 이끌고 프로이센군을 추격하여 섬멸했다. 그의 경력 중에서도 가장 빛나는 것은 1807년의 아일라우 전투에서의 기병 돌격으로, 심각한 위기에 빠져 있던 나폴레옹의 본대를 구원하고 프랑스군의 상황을 일거에 반전시킨 것으로 유명하다.'

In [23]:
corpus[doc_indices[0][0]]

'6일 전역(Campagne des Six-Jours, 1814년 2월 10일~2월 14일)은 나폴레옹 보나파르트가 이끄는 군대가 제6차 대프랑스 동맹군을 상대로 파리가 점령당하기 전까지 벌인 마지막 일련의 승리를 말한다.\n\n황제는 100만에 달하는 동맹군대의 절반에도 못미치는 겨우 70,000명의 군대를 이끌고 전면에 나섰고, 동맹의 몇 개의 주요 군대의 사령관은 야전 원수 게프하르트 레베레히트 폰 블뤼허, 야전 원수 카를 필리프 추 슈바르첸베르크 공작의 지휘를 받으며 다가오고 있었다.\n\n6일 전역은 2월 10일부터 14일에 걸쳐 일어나 샹포베르 전투, 몽미라일 전투, 샤토티에리 전투, 보샹 전투에서 블뤼허 군대는 4번의 주요 패배를 겪었다. 나폴레옹은 12만명의 블뤼허 군대를 자신의 3만명의 군대로 격파해 적에게 17,750명의 부상자를 주었다. 최근의 역사학자와 열광자들은 6일은 황제의 우수한 전역이라고 평가했다.\n\n그러나 황제의 승리는 전체적인 전략적 그림에 어떤 찬스를 만드는데는 충분히 중요하지 않았고, 슈바르첸베레크의 거대한 군대가 서서히 파리를 위협하자 결국 마지막 행진을 끝마치게 되었다.'